In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# E2E ML on GCP: MLOps stage 1 : data management: get started with BigQuery datasets

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_bq_datasets.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
    <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_bq_datasets.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png\" alt="Colab logo"> Run in Colab
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_bq_datasets.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. This tutorial covers stage 1 : data management: get started with BigQuery datasets.

### Dataset

The dataset used for this tutorial is the GSOD dataset from [BigQuery public datasets](https://cloud.google.com/bigquery/public-data). In this version of the dataset you consider the fields year, month and day to predict the value of mean daily temperature (mean_temp).

### Objective

In this tutorial, you learn how to use `BigQuery` as a dataset for training with `Vertex AI`.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Datasets`
- `BigQuery Datasets`

The steps performed include:

- Create a Vertex AI `Dataset` resource from `BigQuery` table -- compatible for `AutoML` training.
- Extract a copy of the dataset from `BigQuery` to a CSV file in Cloud Storage -- compatible for `AutoML` or custom training.
- Select rows from a `BigQuery` dataset into a `pandas` dataframe -- compatible for custom training.
- Select rows from a `BigQuery` dataset into a `tf.data.Dataset` -- compatible for custom training `TensorFlow` models.
- Select rows from extracted CSV files into a `tf.data.Dataset` -- compatible for custom training `TensorFlow` models.
- Create a `BigQuery` dataset from CSV files.
- Extract data from `BigQuery` table into a `DMatrix` -- compatible for custom training `XGBoost` models.

### Recommendations

When doing E2E MLOps on Google Cloud, following are the best practices when dealing with structured (tabular) data in BigQuery:

- For AutoML training:
  - Create a managed dataset with Vertex AI `TabularDataset`.
  - Use the BigQuery table as the input to the dataset.
  - Specify columns and columns transformations when running the AutoML training pipeline job.


- For custom training:
  - For small datasets:
    - Extract the BigQuery to a pandas dataframe.
    - Preprocess the data in the dataframe.
  - For large datasets:
    - TensorFlow model training:
      - Create a tf.data.Dataset generator from the BigQuery table.
      - Specify the columns for the custrom training.
      - Preprocess the data either:
        - Within the generator (upstream)
        - Within the model (downstream)
    - XGBoost model training:
      - Use BigQuery ML built-in XGBoost training.
      - Alternatively, create a DMatrix generator from CSV files extracted from BigQuery table.
    - Pytorch model training:
        - Extract the BigQuery to a pandas dataframe.
        - Preprocess the data in the dataframe.
        - Create a DataLoader generator from the pandas dataframe.


- Alternatively:
    - Extract the BigQuery table to CSV files.
    - Preprocess the CSV files.
    - Create a tf.data.Dataset generator from the CSV files.
    
### Costs
This tutorial uses billable components of Google Cloud:

- Vertex AI
- Cloud Storage
- BigQuery

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing) and [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Installations

Install the following packages to execute this notebook.

In [6]:
!pip install -U xgboost --user

     |████████████████████████████████| 193.7 MB 25 kB/s              


### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
# Get your GCP project id from gcloud
shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = shell_output[0]
print("Project ID:", PROJECT_ID)

Project ID: ma-mx-presales-lab


In [2]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

In [3]:
REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [4]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket


In [5]:
# Already created

BUCKET_URI = "gs://vertex-ai-mlops-bucket" #[your-bucket-name]"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [6]:
#! gsutil mb -l $REGION $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [7]:
#! gsutil ls -al $BUCKET_URI

### Import libraries and define constants

In [8]:
import google.cloud.aiplatform as aiplatform
import pandas as pd
import xgboost as xgb
from google.cloud import bigquery

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [9]:
aiplatform.init(project=PROJECT_ID, location=REGION)

### Create BigQuery client

Create the BigQuery client.

In [10]:
bqclient = bigquery.Client(project=PROJECT_ID)

#### Location of BigQuery training data.

Now, set the variable `IMPORT_FILE` to the location of the data table in BigQuery and `BQ_TABLE` with the table id.

In [11]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"
BQ_TABLE = "bigquery-public-data.samples.gsod"

### 1 Create BigQuery Dataset for AutoML

#### BigQuery input data

Next, create the `Dataset` resource using the `create` method for the `TabularDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `bq_source`: Import data items from a BigQuery table into the `Dataset` resource.
- `labels`: User defined metadata. In this example, you store the location of the Cloud Storage bucket containing the user defined data.

Learn more about [TabularDataset from BigQuery table](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_bigquery_sample-python).

In [14]:
dataset = aiplatform.TabularDataset.create(
    display_name="NOAA historical weather data" + "_" + TIMESTAMP,
    bq_source=[IMPORT_FILE],
    labels={"user_metadata": BUCKET_URI[5:]},
)

label_column = "mean_temp"

print(dataset.resource_name)

INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/825075454589/locations/us-central1/datasets/2682575275308351488/operations/8546273603736305664
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/825075454589/locations/us-central1/datasets/2682575275308351488
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/825075454589/locations/us-central1/datasets/2682575275308351488')
projects/825075454589/locations/us-central1/datasets/2682575275308351488


### 2 Copyign a BigQuery table to a CSV file to Cloud Storage

Next, you make a copy of the BigQuery table as a CSV file, to Cloud Storage using the BigQuery extract command.

Learn more about [BigQuery command line interface](https://cloud.google.com/bigquery/docs/reference/bq-cli-reference).

In [13]:
comps = BQ_TABLE.split(".")
BQ_PROJECT_DATASET_TABLE = comps[0] + ":" + comps[1] + "." + comps[2]

! bq --location=us extract --destination_format CSV $BQ_PROJECT_DATASET_TABLE $BUCKET_URI/BQ_2_CSVs/mydata*.csv

IMPORT_FILES = ! gsutil ls $BUCKET_URI/BQ_2_CSVs/mydata*.csv

print(IMPORT_FILES[-10:])

EXAMPLE_FILE = IMPORT_FILES[0]

! gsutil cat $EXAMPLE_FILE | head

Waiting on bqjob_r276809301982e5e1_0000018068983d0f_1 ... (48s) Current status: DONE   
['gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000105.csv', 'gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000106.csv', 'gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000107.csv', 'gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000108.csv', 'gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000109.csv', 'gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000110.csv', 'gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000111.csv', 'gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000112.csv', 'gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000113.csv', 'gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000114.csv']
station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,num_mean_sealevel_pressure_samples,mean_station_pressure,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_sampl

### Create the Dataset

#### CSV input data

Next, create the `Dataset` resource using the `create` method for the `TabularDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.
- `labels`: User defined metadata. In this example, you store the location of the Cloud Storage bucket containing the user defined data.

Learn more about [TabularDataset from CSV files](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_gcs_sample-python)

In [21]:
IMPORT_FILES[:5]

['gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000000.csv',
 'gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000001.csv',
 'gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000002.csv',
 'gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000003.csv',
 'gs://vertex-ai-mlops-bucket/BQ_2_CSVs/mydata000000000004.csv']

In [22]:
gcs_source = IMPORT_FILES

dataset = aiplatform.TabularDataset.create(
    display_name="NOAA historical weather data" + "_" + TIMESTAMP,
    gcs_source=gcs_source,
    labels={"user_metadata": BUCKET_URI[5:]},
)

label_column = "mean_temp"

print(dataset.resource_name)

INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/825075454589/locations/us-central1/datasets/9158751539467124736/operations/6591711365457510400
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/825075454589/locations/us-central1/datasets/9158751539467124736
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/825075454589/locations/us-central1/datasets/9158751539467124736')
projects/825075454589/locations/us-central1/datasets/9158751539467124736


### Create a view of the BigQuery dataset

Alternatively, you can create a logical view of a BigQuery dataset that has a subset of the fields.

Learn more about [Creating BigQuery views](https://cloud.google.com/bigquery/docs/views).

In [14]:
# Set dataset name and view name in BigQuery
BQ_MY_DATASET = "[your-dataset-name]"
BQ_MY_TABLE = "[your-view-name]"

# Otherwise, use the default names
if (
    BQ_MY_DATASET == ""
    or BQ_MY_DATASET is None
    or BQ_MY_DATASET == "[your-dataset-name]"
):
    BQ_MY_DATASET = "mlops_dataset_" + TIMESTAMP

if BQ_MY_TABLE == "" or BQ_MY_TABLE is None or BQ_MY_TABLE == "[your-view-name]":
    BQ_MY_TABLE = "mlops_view_" + TIMESTAMP

In [15]:
# Create the resources
! bq --location=US mk -d \
$PROJECT_ID:$BQ_MY_DATASET

sql_script = f'''
CREATE OR REPLACE VIEW `{PROJECT_ID}.{BQ_MY_DATASET}.{BQ_MY_TABLE}`
AS SELECT station_number,year,month,day,mean_temp FROM `{BQ_TABLE}`
'''
print(sql_script)

query = bqclient.query(sql_script)

Dataset 'ma-mx-presales-lab:mlops_dataset_20220427005512' successfully created.

CREATE OR REPLACE VIEW `ma-mx-presales-lab.mlops_dataset_20220427005512.mlops_view_20220427005512`
AS SELECT station_number,year,month,day,mean_temp FROM `bigquery-public-data.samples.gsod`



### Read the BigQuery dataset into a pandas dataframe

Next, you read a sample of the dataset into a pandas dataframe using BigQuery `list_rows()` and `to_dataframe()` method, as follows:

- `list_rows()`: Performs a query on the specified table and returns a row iterator to the query results. Optionally specify:
 - `selected_fields`: Subset of fields (columns) to return.
 - `max_results`: The maximum number of rows to return. Same as SQL LIMIT command.


- `rows.to_dataframe()`: Invokes the row iterator and reads in the data into a pandas dataframe.

Learn more about [Loading BigQuery table into a dataframe](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)

In [22]:
# Download the table.
table = bigquery.TableReference.from_string(BQ_TABLE)

rows = bqclient.list_rows(
    table,
    max_results=500,
    selected_fields=[
        bigquery.SchemaField("station_number", "STRING"),
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("month", "INTEGER"),
        bigquery.SchemaField("day", "INTEGER"),
        bigquery.SchemaField("mean_temp", "FLOAT"),
    ],
)

dataframe = rows.to_dataframe()
print(dataframe.head())

  station_number  year  month  day  mean_temp
0          39730  1929     10   20  52.799999
1          33110  1929     12   18  47.500000
2          37770  1931      4   24  50.200001
3         726810  1931      6   23  65.099998
4         726810  1931      3    2  42.799999


### Read the BigQuery dataset into a tf.data.Dataset

Next, you read a sample of the dataset into a tf.data.Dataset using TensorFlow IO `BigQueryClient()` and `read_session()` method, with the following parameters:

- `parent`: Your project ID.
- `project_id`: The project ID of the BigQuery table.
- `dataset_id`: The ID of the BigQuery dataset.
- `table_id`. The ID of the table within the corresponding BigQuery dataset.
- `selected_fields`: Subset of fields (columns) to return.
- `output_types`: The output types of the corresponding fields.
- `requested_streams`: The number of parallel readers.

Learn more about [BigQuery TensorFlow reader](https://www.tensorflow.org/io/tutorials/bigquery).

Learn more about [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

In [15]:
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient

feature_names = "station_number,year,month,day".split(",")

target_name = "mean_temp"

def read_bigquery(project, dataset, table):
    tensorflow_io_bigquery_client = BigQueryClient()
    read_session = tensorflow_io_bigquery_client.read_session(
        parent="projects/" + PROJECT_ID,
        project_id=project,
        dataset_id=dataset,
        table_id=table,
        selected_fields=feature_names + [target_name],
        output_types=[dtypes.string] + [dtypes.int32] * 3 + [dtypes.float32],
        requested_streams=2,
    )

    dataset = read_session.parallel_read_rows()
    return dataset


PROJECT, DATASET, TABLE = IMPORT_FILE.split("/")[-1].split(".")
tf_dataset = read_bigquery(PROJECT, DATASET, TABLE)

print(tf_dataset.take(1))

2022-04-27 01:18:37.758208: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2022-04-27 01:18:37.758670: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA
2022-04-27 01:18:37.870510: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-04-27 01:18:37.870558: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-27 01:18:37.870580: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-7-20220125-120050): /proc/driver/nvidia/version does not exist
2022-04-27 01:18:37.870974

<TakeDataset shapes: OrderedDict([(day, ()), (mean_temp, ()), (month, ()), (station_number, ()), (year, ())]), types: OrderedDict([(day, tf.int32), (mean_temp, tf.float32), (month, tf.int32), (station_number, tf.string), (year, tf.int32)])>


### Read CSV files into a tf.data.Dataset

Alternatively, when your data is in CSV files, you can load the dataset into a tf.data.Dataset using `tf.data.experimental.CsvDataset`, with the following parameters:

- `filenames`: A list of one or more CSV files.
- `header`: Whether CSV file(s) contain a header.
- `select_cols`: Subset of fields (columns) to return.
- `record_defaults`: The output types of the corresponding fields.

Learn more about [tf.data CsvDataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/CsvDataset)

In [20]:
import tensorflow as tf

feature_names = ["station_number,year,month,day".split(",")]

target_name = "mean_temp"

tf_dataset = tf.data.experimental.CsvDataset(
    filenames=IMPORT_FILES,
    header=True,
    select_cols=feature_names.append(target_name),
    record_defaults=[dtypes.string] + [dtypes.int32] * 3 + [dtypes.float32],
)

print(tf_dataset.take(1))

<TakeDataset shapes: ((), (), (), (), ()), types: (tf.string, tf.int32, tf.int32, tf.int32, tf.float32)>


### Create a BigQuery dataset from a pandas dataframe

You can create a BigQuery dataset from a pandas dataframe using the BigQuery `create_dataset()` and `load_table_from_dataframe()` methods, as follows:

- `create_dataset()`: Creates an empty BigQuery dataset, with the following parameters:
 - `dataset_ref`: The `DatasetReference` created from the dataset_id -- e.g., samples.
- `load_table_from_dataframe()`: Loads one or more CSV files into a table within the corresponding dataset, with the following parameters:
 - `dataframe`: The dataframe.
 - `table`: The `TableReference` for the table.
 - `job_config`: Specifications on how to load the dataframe data.

In [23]:
LOCATION = "us"

SCHEMA = [
    bigquery.SchemaField("station_number", "STRING"),
    bigquery.SchemaField("year", "INTEGER"),
    bigquery.SchemaField("month", "INTEGER"),
    bigquery.SchemaField("day", "INTEGER"),
    bigquery.SchemaField("mean_temp", "FLOAT"),
]

DATASET_ID = "samples"
TABLE_ID = "gsod"

def create_bigquery_dataset(dataset_id):
    dataset = bigquery.Dataset(
        bigquery.dataset.DatasetReference(PROJECT_ID, dataset_id)
    )
    dataset.location = "us"

    try:
        dataset = bqclient.create_dataset(dataset)  # API request
        return True
    except Exception as err:
        print(err)
        if err.code != 409:  # http_client.CONFLICT
            raise
    return False


def load_data_into_bigquery(dataframe, dataset_id, table_id):
    create_bigquery_dataset(dataset_id)
    dataset = bqclient.dataset(dataset_id)
    table = dataset.table(table_id)

    job_config = bigquery.LoadJobConfig(
        # Specify a (partial) schema. All columns are always written to the
        # table. The schema is used to assist in data type definitions.
        schema=[
            bigquery.SchemaField("station_number", "STRING"),
            bigquery.SchemaField("year", "INTEGER"),
            bigquery.SchemaField("month", "INTEGER"),
            bigquery.SchemaField("day", "INTEGER"),
            bigquery.SchemaField("mean_temp", "FLOAT"),
        ],
        # Optionally, set the write disposition. BigQuery appends loaded rows
        # to an existing table by default, but with WRITE_TRUNCATE write
        # disposition it replaces the table with the loaded data.
        write_disposition="WRITE_TRUNCATE",
    )

    NEW_BQ_TABLE = f"{PROJECT_ID}.{dataset_id}.{table_id}"

    job = bqclient.load_table_from_dataframe(
        dataframe, NEW_BQ_TABLE, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = bqclient.get_table(NEW_BQ_TABLE)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), NEW_BQ_TABLE
        )
    )


load_data_into_bigquery(dataframe, DATASET_ID, TABLE_ID)

Loaded 500 rows and 5 columns to ma-mx-presales-lab.samples.gsod


### Create a BigQuery dataset from CSV files

You can create a BigQuery dataset from CSV files using the BigQuery `create_dataset()` and `load_table_from_uri()` methods, as follows:

- `create_dataset()`: Creates an empty BigQuery dataset, with the following parameters:
 - `dataset_ref`: The `DatasetReference` created from the dataset_id -- e.g., samples.
- `load_table_from_uri()`: Loads one or more CSV files into a table within the corresponding dataset, with the following parameters:
 - `url`: A set of one or more CVS files in Cloud Storage storage.
 - `table`: The `TableReference` for the table.
 - `job_config`: Specifications on how to load the CSV data.

Learn more about [Importing CSV data into BigQuery](https://www.tensorflow.org/io/tutorials/bigquery#import_census_data_into_bigquery).

In [24]:
LOCATION = "us"

CSV_SCHEMA = [
    bigquery.SchemaField("station_number", "STRING"),
    bigquery.SchemaField("wban_number", "STRING"),
    bigquery.SchemaField("year", "INTEGER"),
    bigquery.SchemaField("month", "INTEGER"),
    bigquery.SchemaField("day", "INTEGER"),
    bigquery.SchemaField("mean_temp", "FLOAT"),
    bigquery.SchemaField("num_mean_temp_samples", "INTEGER"),
    bigquery.SchemaField("mean_dew_point", "FLOAT"),
    bigquery.SchemaField("num_mean_dew_point_samples", "INTEGER"),
    bigquery.SchemaField("mean_sealevel_pressure", "FLOAT"),
    bigquery.SchemaField("num_mean_sealevel_pressure_samples", "INTEGER"),
    bigquery.SchemaField("mean_station_pressure", "FLOAT"),
    bigquery.SchemaField("num_mean_station_pressure_samples", "INTEGER"),
    bigquery.SchemaField("mean_visibility", "FLOAT"),
    bigquery.SchemaField("num_mean_visibility_samples", "INTEGER"),
    bigquery.SchemaField("mean_wind_speed", "FLOAT"),
    bigquery.SchemaField("num_mean_wind_speed_samples", "INTEGER"),
    bigquery.SchemaField("max_sustained_wind_speed", "FLOAT"),
    bigquery.SchemaField("max_gust_wind_speed", "FLOAT"),
    bigquery.SchemaField("max_temperature", "FLOAT"),
    bigquery.SchemaField("max_temperature_explicit", "BOOLEAN"),
    bigquery.SchemaField("min_temperature", "FLOAT"),
    bigquery.SchemaField("min_temperature_explicit", "BOOLEAN"),
    bigquery.SchemaField("total_percipitation", "FLOAT"),
    bigquery.SchemaField("snow_depth", "FLOAT"),
    bigquery.SchemaField("fog", "BOOLEAN"),
    bigquery.SchemaField("rain", "BOOLEAN"),
    bigquery.SchemaField("snow", "BOOLEAN"),
    bigquery.SchemaField("hail", "BOOLEAN"),
    bigquery.SchemaField("thunder", "BOOLEAN"),
    bigquery.SchemaField("tornado", "BOOLEAN"),
]


DATASET_ID = "samples"
TABLE_ID = "gsod"


def load_data_into_bigquery(url, dataset_id, table_id):
    create_bigquery_dataset(dataset_id)
    dataset = bqclient.dataset(dataset_id)
    table = dataset.table(table_id)

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.schema = CSV_SCHEMA
    job_config.skip_leading_rows = 1  # heading

    load_job = bqclient.load_table_from_uri(url, table, job_config=job_config)
    print("Starting job {}".format(load_job.job_id))

    load_job.result()  # Waits for table load to complete.
    print("Job finished.")

    destination_table = bqclient.get_table(table)
    print("Loaded {} rows.".format(destination_table.num_rows))


load_data_into_bigquery(IMPORT_FILES, DATASET_ID, TABLE_ID)

409 POST https://bigquery.googleapis.com/bigquery/v2/projects/ma-mx-presales-lab/datasets?prettyPrint=false: Already Exists: Dataset ma-mx-presales-lab:samples
Starting job abfdea63-e9e0-432b-90f7-020ce81b016d
Job finished.
Loaded 114420316 rows.


### Read BigQuery table into XGboost DMatrix

Currently, there is no direct data feeding connector between BigQuery and the open source XGBoost. The BigQuery ML service has a built-in XGBoost training module.

Alernatively, you extract the data either as a pandas dataframe or as CSV files. The extracted data is then given as an input to a `DMatrix` object when training the model.

Learn more about [Getting started with built-in XGBoost](https://cloud.google.com/ai-platform/training/docs/algorithms/xgboost-start).

### Read pandas table into XGboost DMatrix

Next, you load the pandas dataframe into a `DMatrix` object. XGBoost does not support non-numeric inputs. Any column that is categorical need to be one-hot encoded prior to loading the dataframe.

In [ ]:
dataframe["station_number"] = pd.to_numeric(dataframe["station_number"])
labels = dataframe["mean_temp"]
data = dataframe.drop(4)

dtrain = xgb.DMatrix(data, label=labels)

### Read CSV files into XGboost DMatrix

Currently, there is no Cloud Storage support in XGBoost. If you use CSV files for input, you need to download them locally.

In [ ]:
! gsutil cp $EXAMPLE_FILE data.csv

dtrain = xgb.DMatrix("data.csv?format=csv&label_column=4")

# Clean up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Vertex AI Dataset resource
- Cloud Storage Bucket
- BigQuery Dataset

Set `delete_storage` to _True_ to delete the storage resources used in this notebook.

In [ ]:
import os

# Delete the dataset using the Vertex dataset object
dataset.delete()
# Delete the temporary BigQuery dataset
! bq rm -r -f $PROJECT_ID:$DATASET_ID

delete_storage = False
if delete_storage or os.getenv("IS_TESTING"):
    # Delete the created GCS bucket
    ! gsutil rm -r $BUCKET_URI
    # Delete the created BigQuery datasets
    ! bq rm -r -f $PROJECT_ID:$BQ_MY_DATASET